| <img src ='https://bit.ly/39XPeCc' width=200 align='left'> |
|:--|
|<h1>    Random_forest_HB.ipynb 
</h1> * 현범님 앤덤포래스트 노트북
* 랜덤포레스트 샘플
* 처음부터 하나씩 해보는 Sklearn

In [1]:
""" 
# 기본적인 module import 위치로 작업폴더 변경 getcwd() --> chidr()
# HOME 을 지정하고 작업폴더를 HOME 으로 변경.
"""
import os

HOME= 'dacon_competition_2020'
dir_base = "".join(os.getcwd().partition(HOME)[:2]) + "\\"
os.chdir(dir_base)    

## 사용함수의 정의

In [2]:
from _assets.config_dirs import *
from _assets.modules import *


# 화일분석에 필요한 공동폴더를 등록합니다.
- asset = dir_base + '_assets/'
- statics dir_base + '_statics/'

#  echo = True : SYS.PATH INSERT 상황 보여줌


# OS 화일 및 DF 정보조회를 위한 탐색 모듈



In [3]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectFromModel

In [2]:
# train = pd.read_feather('./data/train.feature')
# test = pd.read_feather('./data/test.feature')
# sample = pd.read_feather('./data/sample_submission.feather')

In [33]:
"""
def DataPreProceccing(train):
    df_train = pd.DataFrame(columns=['game_id'])
    df_train.game_id = train.game_id.unique()
    
    mask = train.shift(-1).game_id != train.game_id
    df_train['time']=np.array(train[mask].time)
    
    df = train[train.player==0]
    df_train['player0_species']=np.array(df[df.shift(-1).game_id != df.game_id].species)
    
    df = train[train.player==1]
    df_train['player1_species']=np.array(df[df.shift(-1).game_id != df.game_id].species)
    
    df = train[train.player==0]
    df_train['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
    
    df = train[train.player==1]
    df_train['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
    
    for event in train.event.unique():
        df = train[(train.player==0)&(train.event==event)]
        df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
        df['game_id']= np.array(df.index)
        df_train = pd.merge(df_train, df, on='game_id', how='left')

        df = train[(train.player==1)&(train.event==event)]
        df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
        df['game_id']= np.array(df.index)
        df_train = pd.merge(df_train, df, on='game_id', how='left')
        
    df_train = df_train.fillna(0)
    
    for event in train.event.unique():
        df_train[event + '_diff'] = df_train[event+'_count_0'] - df_train[event+'_count_1']
        
    #df_train['winner'] = np.array(train[train.shift(-1).game_id != train.game_id].winner)

    idx = df_train[df_train.player0_species=='T'].index
    df_train.loc[idx, 'player0_species'] = 0
    
    idx = df_train[df_train.player0_species=='P'].index
    df_train.loc[idx, 'player0_species'] = 1
    
    idx = df_train[df_train.player0_species=='Z'].index
    df_train.loc[idx, 'player0_species'] = 2

    idx = df_train[df_train.player1_species=='T'].index
    df_train.loc[idx, 'player1_species'] = 0
    
    idx = df_train[df_train.player1_species=='P'].index
    df_train.loc[idx, 'player1_species'] = 1
    
    idx = df_train[df_train.player1_species=='Z'].index
    df_train.loc[idx, 'player1_species'] = 2 
    
    return df_train
"""    

In [4]:
show_ls(dir_base + remake)

False
DIR_TARGET=C:\Users\nitt0\Documents\Github\dacon_competition_2020
C:\Users\nitt0\Documents\Github\dacon_competition_2020\c03_starcraft_prediction/data_remake/
----------------------------------------
  01. .ipynb_checkpoints
  02. accumujated_winning_rate_per_time.csv
  03. accumulated_winning_rate_per_time.csv
  04. df_new_rate04_correct.csv
  05. df_win_rate_every_10sec.csv
  06. df_win_rate_every_20sec.csv
  07. df_win_rate_every_40sec.csv
  08. df_xtest_remake.csv
  09. df_xtest_remake_final.csv
  10. df_xtrain_remake.csv
  11. df_xtrain_remake_final.csv
  12. df_ytrain_remake.csv
  13. xtra_remake_xtest.csv
  14. xtra_remake_xtest_r01.csv
  15. xtra_remake_xtrain.csv
  16. xtra_remake_xtrain_r01.csv


In [5]:
df_train = pd.read_csv(dir_base + remake + 'df_xtrain_remake.csv')
df_train.describe()

,game_id,P0_species,P0_Ability,P0_AddToControlGroup,P0_Camera,P0_ControlGroup,P0_GetControlGroup,P0_Right Click,P0_Selection,P0_SetControlGroup,...,P1_Selection,P1_SetControlGroup,delta_Ability,delta_AddToControlGroup,delta_Camera,delta_ControlGroup,delta_GetControlGroup,delta_Right Click,delta_Selection,delta_SetControlGroup
count,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,...,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000
mean,19435.500000,0.943790,48.806467,1.918450,407.357918,0.396275,129.523307,148.942118,122.933242,4.365610,...,122.131560,4.399156,-0.102387,-0.004939,1.034858,0.002290,0.475406,0.346753,0.801682,-0.033546
std,11221.524169,0.846693,28.119083,3.038626,222.351156,3.431973,186.127982,97.965154,85.964370,4.783873,...,85.485426,4.799611,18.469226,3.996941,214.550025,4.861978,219.333636,87.614048,95.411569,5.689446
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-129.000000,-27.000000,-1293.000000,-271.000000,-1393.000000,-785.000000,-807.000000,-56.000000
25%,9717.750000,0.000000,27.000000,0.000000,238.000000,0.000000,11.000000,71.000000,60.000000,1.000000,...,60.000000,1.000000,-9.000000,-2.000000,-117.000000,0.000000,-57.250000,-47.000000,-47.000000,-3.000000
50%,19435.500000,1.000000,46.000000,0.000000,390.000000,0.000000,45.000000,133.000000,102.000000,3.000000,...,102.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,29153.250000,2.000000,66.000000,3.000000,549.000000,0.000000,175.000000,210.000000,167.000000,6.000000,...,165.000000,6.000000,9.000000,2.000000,121.000000,0.000000,58.000000,48.000000,49.000000,3.000000
max,38871.000000,2.000000,216.000000,35.000000,1832.000000,241.000000,2320.000000,856.000000,953.000000,62.000000,...,960.000000,73.000000,141.000000,35.000000,1589.000000,241.000000,2222.000000,626.000000,733.000000,57.000000


In [6]:
df_train.shape

(38872, 27)

In [30]:
"""
# y = dsklearn.feature_selection,-1]
# X = df_train.iloc[:,:-1]
# (31097, 30) --> (31097, 27) 왜 피처가 3개출어 들었나?
"""
X = df_train
y = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv', index_col='Unnamed: 0')
y = y['winner']           # pandas.core.series.Series

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((31097, 27), (7775, 27), (31097,), (7775,))

## 중요 피쳐 뽑아보기

In [32]:
forest = RandomForestClassifier(n_estimators=1000, random_state=1997, n_jobs=-1)


In [33]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=1997, verbose=0,
                       warm_start=False)

In [34]:
importances = forest.feature_importances_
importances

array([0.05402026, 0.01132358, 0.04555251, 0.01903538, 0.05202668,
       0.00396083, 0.04616396, 0.04895633, 0.04955728, 0.02721587,
       0.01124108, 0.04614369, 0.01868508, 0.05177143, 0.00378693,
       0.04481305, 0.04927326, 0.04933454, 0.02720883, 0.06421215,
       0.02792735, 0.05137017, 0.00723228, 0.0537872 , 0.05096644,
       0.05076556, 0.03366827])

In [35]:
indices = np.argsort(importances)[::-1]
indices

array([19,  0, 23,  4, 13, 21, 24, 25,  8, 17, 16,  7,  6, 11,  2, 15, 26,
       20,  9, 18,  3, 12,  1, 10, 22,  5, 14], dtype=int64)

In [36]:
feat_labels = df_train.columns[1:]
feat_labels

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" %(f+1,30,feat_labels[indices[f]],importances[indices[f]]))

 1) delta_AddToControlGroup        0.064212
 2) P0_species                     0.054020
 3) delta_Right Click              0.053787
 4) P0_ControlGroup                0.052027
 5) P1_ControlGroup                0.051771
 6) delta_ControlGroup             0.051370
 7) delta_Selection                0.050966
 8) delta_SetControlGroup          0.050766
 9) P0_SetControlGroup             0.049557
10) P1_SetControlGroup             0.049335
11) P1_Selection                   0.049273
12) P0_Selection                   0.048956
13) P0_Right Click                 0.046164
14) P1_AddToControlGroup           0.046144
15) P0_AddToControlGroup           0.045553
16) P1_Right Click                 0.044813


IndexError: index 26 is out of bounds for axis 0 with size 26

In [27]:
sfm = SelectFromModel(forest, threshold=0.03, prefit=True)

In [28]:
X_selected = sfm.transform(X_train)

In [30]:
forest = RandomForestClassifier(n_estimators=1000, random_state=1997, n_jobs=-1)

forest.fit(X_selected, y_train)


X_test_selected = sfm.transform(X_test)

pred = forest.predict(X_test_selected)

err = mean_absolute_error(y_test, pred)
err

0.41672025723472667

In [34]:
x_infer = DataPreProceccing(test)


In [36]:
x_infer_selected = sfm.transform(x_infer)

In [37]:
y_infer = forest.predict_proba(x_infer_selected)

In [39]:
y_infer

array([[0.448, 0.552],
       [0.529, 0.471],
       [0.53 , 0.47 ],
       ...,
       [0.516, 0.484],
       [0.461, 0.539],
       [0.561, 0.439]])

In [42]:
sample['winner'] = y_infer

In [43]:
sample

,game_id,winner
0,38872,0.448
1,38873,0.529
2,38874,0.530
3,38875,0.565
4,38876,0.497
...,...,...
16782,55654,0.504
16783,55655,0.529
16784,55656,0.516
16785,55657,0.461


In [44]:
sample.to_csv('./data/HB_submission_RNF20200401.csv', index=False)